# Analysis of Belle2 Data

In [1]:
import os, sys, pprint
import glob
from root_pandas import read_root
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing

Welcome to JupyROOT 6.16/00


/home/philipp/anaconda3/envs/BelleAnalysis/lib/python3.6/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.16.4 is currently installed but you installed root_numpy against numpy 1.9.3. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)


In [2]:
!ls -l ../DATA/
!pwd

total 56352
-rwxr--r--  1 philipp philipp   584128 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_ccbar_0.root
-rwxr--r--  1 philipp philipp   117621 May 14 14:58 B0_etapr-eta-gg2pi_KS-pi+pi-_output_charged_17.root
-rwxr--r--  1 philipp philipp   155084 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_0.root
-rwxr--r--  1 philipp philipp   177149 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_1.root
-rwxr--r--  1 philipp philipp   183977 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_2.root
-rwxr--r--  1 philipp philipp   176779 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_3.root
-rwxr--r--  1 philipp philipp   185028 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_4.root
-rwxr--r--  1 philipp philipp   141664 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_5.root
-rwxr--r--  1 philipp philipp   192110 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_6.root
-rwxr--r--  1 philipp philipp   179731 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_7.root

## Reading and Prepairing Data

In [3]:
path = '/home/philipp/Desktop/Project/DATA/'
SFiles = glob.glob(os.path.join(path, 'Signal/*.root'))
CFiles = glob.glob(os.path.join(path, 'Continuous/*.root'))
PFiles = glob.glob(os.path.join(path, 'Peaking/*.root'))

Full_Signal = pd.concat((read_root(f, 'B0') for f in SFiles))
Full_Continuous = pd.concat((read_root(f, 'B0') for f in CFiles))
Full_Peaking = pd.concat((read_root(f, 'B0') for f in PFiles))

/home/philipp/anaconda3/envs/BelleAnalysis/lib/python3.6/site-packages/root_pandas/readwrite.py:320: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  df = df.reindex_axis(columns, axis=1, copy=False)


In [4]:
Full_Continuous

,exp_no,run_no,evt_no,B0_K_S0_M,B0_K_S0_ErrM,B0_K_S0_SigM,B0_M,B0_ErrM,B0_SigM,B0_etap_M,...,B0_K_S0_TruthVtxProd,B0_TruthTagVx,B0_TruthTagVy,B0_TruthTagVz,B0_mcTagPDG,B0_TruthDeltaT,B0_CSMVA,B0_TrCSMVA,nCands,iCand
0,0,0,51571101,0.500895,0.002853,1.149850,5.135701,0.012654,-11.366517,0.938931,...,"[0.0019425296, -0.0031857349, 0.005766661]",-111.0,-111.0,-111.0,0,14707.006836,0.261846,-1.036396,1,0
1,0,0,51572506,0.496650,0.002876,-0.335105,5.268522,0.019502,-0.564453,0.916602,...,"[-0.0017205713, 0.0036101926, 0.020706093]",-111.0,-111.0,-111.0,0,14707.006836,0.134443,-1.862235,2,0
2,0,0,51572506,0.496650,0.002876,-0.335105,5.184201,0.021533,-4.427056,0.924155,...,"[-0.0017205713, 0.0036101926, 0.020706093]",-111.0,-111.0,-111.0,0,14707.006836,0.096710,-2.234329,2,1
3,0,0,52815187,0.512523,0.003407,4.376027,5.264990,0.018447,-0.788202,0.952054,...,"[-0.0011433143, 1.0134625e-06, -0.00859407]",-111.0,-111.0,-111.0,0,14707.006836,0.155727,-1.690369,1,0
4,0,0,52818414,0.499365,0.003113,0.562306,5.200452,0.021283,-3.715473,0.939792,...,"[0.00070354354, -3.8584048e-07, -0.006386356]",-111.0,-111.0,-111.0,0,14707.006836,0.009158,-4.683872,1,0
5,0,0,52438085,0.507719,0.004798,2.106009,5.424548,0.042539,3.409029,0.922027,...,"[7.33112e-05, -2.2381237e-06, 0.016870165]",-111.0,-111.0,-111.0,0,14707.006836,0.003701,-5.595561,4,0
6,0,0,52438085,0.507719,0.004798,2.106009,5.441957,0.042845,3.791087,0.913086,...,"[7.33112e-05, -2.2381237e-06, 0.016870165]",-111.0,-111.0,-111.0,0,14707.006836,0.003192,-5.743785,4,1
7,0,0,52438085,0.515977,0.005975,3.073336,5.410072,0.032874,3.970957,0.922027,...,"[7.33112e-05, -2.2381237e-06, 0.016870165]",-111.0,-111.0,-111.0,0,14707.006836,0.006637,-5.008420,4,2
8,0,0,52438085,0.515977,0.005975,3.073336,5.427326,0.033301,4.438130,0.913086,...,"[7.33112e-05, -2.2381237e-06, 0.016870165]",-111.0,-111.0,-111.0,0,14707.006836,0.007339,-4.907236,4,3
9,0,0,117041010,0.506843,0.033634,0.274386,5.316215,0.124951,0.293591,0.949488,...,"[0.0, 0.0, 0.0]",-111.0,-111.0,-111.0,0,14707.006836,0.515877,0.063531,1,0


__Branches, organized by Categories__

In [5]:
General = ['exp_no', 'run_no', 'evt_no', 'nCands', 'iCand']
Masses = ['B0_M', 'B0_ErrM', 'B0_SigM', 'B0_K_S0_M', 'B0_K_S0_ErrM', 'B0_K_S0_SigM', 'B0_etap_M', 'B0_etap_ErrM', 
          'B0_etap_SigM', 'B0_etap_eta_M', 'B0_etap_eta_ErrM', 'B0_etap_eta_SigM']
Kinetics = ['B0_P', 'B0_P4', 'B0_deltae', 'B0_mbc', 'B0_etap_P', 'B0_etap_P4', 
            'B0_etap_eta_P', 'B0_etap_eta_P4', 'B0_etap_eta_gamma0_P', 'B0_etap_eta_gamma0_P4',
            'B0_etap_eta_gamma1_P', 'B0_etap_eta_gamma1_P4', 'B0_etap_pi0_P', 'B0_etap_pi0_P4',
            'B0_etap_pi1_P', 'B0_etap_pi1_P4', 'B0_K_S0_P', 'B0_K_S0_P4']
Kinetics_comp = ['B0_P', 'B0_P40', 'B0_P41', 'B0_P42', 'B0_P43', 'B0_deltae', 'B0_mbc',
                 'B0_etap_P', 'B0_etap_P40', 'B0_etap_P41', 'B0_etap_P42','B0_etap_P43','B0_etap_eta_P',
                 'B0_etap_eta_P40', 'B0_etap_eta_P41', 'B0_etap_eta_P42', 'B0_etap_eta_P43',
                 'B0_etap_eta_gamma0_P', 'B0_etap_eta_gamma0_P40', 'B0_etap_eta_gamma0_P41', 
                 'B0_etap_eta_gamma0_P42', 'B0_etap_eta_gamma0_P43', 'B0_etap_eta_gamma1_P', 'B0_etap_eta_gamma1_P40',
                 'B0_etap_eta_gamma1_P41', 'B0_etap_eta_gamma1_P42', 'B0_etap_eta_gamma1_P43', 'B0_etap_pi0_P',
                 'B0_etap_pi0_P40', 'B0_etap_pi0_P41', 'B0_etap_pi0_P42', 'B0_etap_pi0_P43', 'B0_etap_pi1_P',
                 'B0_etap_pi1_P40', 'B0_etap_pi1_P41', 'B0_etap_pi1_P42', 'B0_etap_pi1_P43', 'B0_K_S0_P',
                 'B0_K_S0_P40', 'B0_K_S0_P41', 'B0_K_S0_P42', 'B0_K_S0_P43']
FourMomenta = ['B0_P4', 'B0_etap_P4', 'B0_etap_eta_P4', 'B0_etap_eta_gamma0_P4', 'B0_etap_eta_gamma1_P4', 
            'B0_etap_pi0_P4', 'B0_etap_pi1_P4', 'B0_K_S0_P4']
DecayAngles = ['B0_decayAngle__bo0__bc', 'B0_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo0__bc',
               'B0_etap_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo2__bc']
Positions = ['B0_X', 'B0_ErrX', 'B0_Y', 'B0_ErrY', 'B0_Z', 'B0_ErrZ', 'B0_Rho',  
             'B0_etap_X', 'B0_etap_ErrX', 'B0_etap_Y', 'B0_etap_ErrY', 
             'B0_etap_Z', 'B0_etap_ErrZ', 'B0_etap_Rho',
             'B0_etap_eta_X', 'B0_etap_eta_ErrX', 'B0_etap_eta_Y',
             'B0_etap_eta_ErrY', 'B0_etap_eta_Z', 'B0_etap_eta_ErrZ', 'B0_etap_eta_Rho',
             'B0_etap_pi0_X', 'B0_etap_pi0_ErrX', 'B0_etap_pi0_Y', 'B0_etap_pi0_ErrY', 
             'B0_etap_pi0_Z', 'B0_etap_pi0_ErrZ', 'B0_etap_pi0_Rho', 
             'B0_etap_pi1_X', 'B0_etap_pi1_ErrX', 'B0_etap_pi1_Y', 'B0_etap_pi1_ErrY', 
             'B0_etap_pi1_Z', 'B0_etap_pi1_ErrZ', 'B0_etap_pi1_Rho', 
             'B0_K_S0_X', 'B0_K_S0_ErrX', 'B0_K_S0_Y', 'B0_K_S0_ErrY', 'B0_K_S0_Z',
             'B0_K_S0_ErrZ', 'B0_K_S0_Rho', 
             'B0_cosAngleBetweenMomentumAndVertexVector', 'B0_distance', 'B0_significanceOfDistance',
             'B0_dr', 'B0_etap_pi0_dr', 'B0_etap_pi1_dr', 'B0_K_S0_dr']
Vertex = ['B0_VtxPvalue', 'B0_VtxProd', 'B0_VtxProdCov', 'B0_etap_VtxPvalue', 'B0_etap_VtxProd', 'B0_etap_VtxProdCov',
          'B0_etap_eta_VtxPvalue', 'B0_etap_eta_VtxProd', 'B0_etap_VtxProdCov', 
          'B0_etap_pi0_VtxPvalue', 'B0_etap_pi0_VtxProd', 'B0_etap_pi0_VtxProdCov', 
          'B0_etap_pi1_VtxPvalue', 'B0_etap_pi1_VtxProd', 'B0_etap_pi1_VtxProdCov', 
          'B0_K_S0_VtxPvalue', 'B0_K_S0_VtxProd', 'B0_K_S0_VtxProdCov']
Vertex_Training = ['B0_VtxPvalue', 'B0_etap_VtxPvalue', 'B0_etap_eta_VtxPvalue', 'B0_etap_pi0_VtxPvalue',
                   'B0_etap_pi1_VtxPvalue', 'B0_K_S0_VtxPvalue', ]
Continuum_Supression = ['B0_CSMVA', 'B0_TrCSMVA']

Filter = Masses + Kinetics + DecayAngles + Positions + Vertex + Continuum_Supression
Training = Masses + Kinetics_comp + DecayAngles + Positions + Vertex_Training + Continuum_Supression

In [6]:
Training

['B0_M',
 'B0_ErrM',
 'B0_SigM',
 'B0_K_S0_M',
 'B0_K_S0_ErrM',
 'B0_K_S0_SigM',
 'B0_etap_M',
 'B0_etap_ErrM',
 'B0_etap_SigM',
 'B0_etap_eta_M',
 'B0_etap_eta_ErrM',
 'B0_etap_eta_SigM',
 'B0_P',
 'B0_P40',
 'B0_P41',
 'B0_P42',
 'B0_P43',
 'B0_deltae',
 'B0_mbc',
 'B0_etap_P',
 'B0_etap_P40',
 'B0_etap_P41',
 'B0_etap_P42',
 'B0_etap_P43',
 'B0_etap_eta_P',
 'B0_etap_eta_P40',
 'B0_etap_eta_P41',
 'B0_etap_eta_P42',
 'B0_etap_eta_P43',
 'B0_etap_eta_gamma0_P',
 'B0_etap_eta_gamma0_P40',
 'B0_etap_eta_gamma0_P41',
 'B0_etap_eta_gamma0_P42',
 'B0_etap_eta_gamma0_P43',
 'B0_etap_eta_gamma1_P',
 'B0_etap_eta_gamma1_P40',
 'B0_etap_eta_gamma1_P41',
 'B0_etap_eta_gamma1_P42',
 'B0_etap_eta_gamma1_P43',
 'B0_etap_pi0_P',
 'B0_etap_pi0_P40',
 'B0_etap_pi0_P41',
 'B0_etap_pi0_P42',
 'B0_etap_pi0_P43',
 'B0_etap_pi1_P',
 'B0_etap_pi1_P40',
 'B0_etap_pi1_P41',
 'B0_etap_pi1_P42',
 'B0_etap_pi1_P43',
 'B0_K_S0_P',
 'B0_K_S0_P40',
 'B0_K_S0_P41',
 'B0_K_S0_P42',
 'B0_K_S0_P43',
 'B0_decayAngle__

#### Choosing the best candidates in each event

Combinatorics in the reconstruction process often leads to events with multiple candidates, where we carefully have to choose the best without introducing significant bias. To do this we have two possible methods: 

- Choosing the candidate with the reconstructed mass closest to the nominal value
- Choosing the candidate with the best vertex fit

If we were only using the mass we would bias our data towards a very narrow mass peak in cases where two candidates are close to the nominal value. Therefore I suggest using the masses for a generous cut (e.g. $5\sigma$) and then using vertex information to pick the best candidate. 

##### Masses
Nominal masses with errors in GeV $ [M_{nominal}, \sigma_{M_{nominal}}] $ according to the [Particle Data Group](http://pdg.lbl.gov/2019/listings/contents_listings.html)

In events with more than one candidate we can use this to perform a rough cut on candidates requirering, that 

$$ 
M_{reconstructed} \in 
    \left[ M_{nominal} - 5 \times \left( \sigma_{M_{nominal}} + \sigma_{M_{reconstructed}} \right), 
           M_{nominal} + 5 \times \left( \sigma_{M_{nominal}} + \sigma_{M_{reconstructed}} \right) \right]
$$



In [7]:
M_B0 = [5.27955, 0.00026]
M_K0 = [0.497611, 0.000013]
M_etap = [0.95778, 0.00006]
M_eta = [0.547862, 0.000017]

In [8]:
#Signal = Signal[abs(Signal['B0_M'] - M_B0[0]) < 5 * Signal['B0_ErrM']]
#Signal = Signal[abs(Signal['B0_K_S0_M'] - M_K0[0]) < 5 * Signal['B0_K_S0_ErrM']]
#Signal = Signal[abs(Signal['B0_etap_M'] - M_etap[0]) < 5 * Signal['B0_etap_ErrM']]
#Signal = Signal[abs(Signal['B0_etap_eta_M'] - M_eta[0]) < 5 * Signal['B0_etap_eta_ErrM']]

##### Vertex

I need to take a closer look at the vertex information to decide how to use it

In [9]:
# Do stuff

#### Filtering the Signal and chosing variables to train
- We filter the Signal, as we are only interested in correctly reconstructed events
- We choose variables for the training
- We convert arrays to 4 variables

In [10]:
for M in FourMomenta: 
    Full_Signal[M+'0'], Full_Signal[M+'1'], Full_Signal[M+'2'], Full_Signal[M+'3'] = zip(*Full_Signal.pop(M))
    Full_Continuous[M+'0'], Full_Continuous[M+'1'], Full_Continuous[M+'2'], Full_Continuous[M+'3'] = zip(*Full_Continuous.pop(M))
    Full_Peaking[M+'0'], Full_Peaking[M+'1'], Full_Peaking[M+'2'], Full_Peaking[M+'3'] = zip(*Full_Peaking.pop(M))

In [11]:
Pure_Signal = Full_Signal[Full_Signal['B0_isSignal']==1].reset_index(drop=True)

Train_Signal = Pure_Signal[Training]
Train_Continuous = Full_Continuous[Training]
Train_Peaking = Full_Peaking[Training]

#Signal_comp = pd.concat([pd.DataFrame(Signal_Pure[c].tolist()).add_prefix(c[:c.index('4')]).add_suffix(c[c.index('4')+1:]) if '4' in c else Signal_Pure[c] for c in Signal_Pure], axis=1)
#Background_comp = pd.concat([pd.DataFrame(Background[c].tolist()).add_prefix(c[:c.index('4')]).add_suffix(c[c.index('4')+1:]) if '4' in c else Background[c] for c in Background], axis=1)

In [12]:
from IPython.display import display
pd.options.display.max_columns = None
display(Train_Signal)

,B0_M,B0_ErrM,B0_SigM,B0_K_S0_M,B0_K_S0_ErrM,B0_K_S0_SigM,B0_etap_M,B0_etap_ErrM,B0_etap_SigM,B0_etap_eta_M,B0_etap_eta_ErrM,B0_etap_eta_SigM,B0_P,B0_P40,B0_P41,B0_P42,B0_P43,B0_deltae,B0_mbc,B0_etap_P,B0_etap_P40,B0_etap_P41,B0_etap_P42,B0_etap_P43,B0_etap_eta_P,B0_etap_eta_P40,B0_etap_eta_P41,B0_etap_eta_P42,B0_etap_eta_P43,B0_etap_eta_gamma0_P,B0_etap_eta_gamma0_P40,B0_etap_eta_gamma0_P41,B0_etap_eta_gamma0_P42,B0_etap_eta_gamma0_P43,B0_etap_eta_gamma1_P,B0_etap_eta_gamma1_P40,B0_etap_eta_gamma1_P41,B0_etap_eta_gamma1_P42,B0_etap_eta_gamma1_P43,B0_etap_pi0_P,B0_etap_pi0_P40,B0_etap_pi0_P41,B0_etap_pi0_P42,B0_etap_pi0_P43,B0_etap_pi1_P,B0_etap_pi1_P40,B0_etap_pi1_P41,B0_etap_pi1_P42,B0_etap_pi1_P43,B0_K_S0_P,B0_K_S0_P40,B0_K_S0_P41,B0_K_S0_P42,B0_K_S0_P43,B0_decayAngle__bo0__bc,B0_decayAngle__bo1__bc,B0_etap_decayAngle__bo0__bc,B0_etap_decayAngle__bo1__bc,B0_etap_decayAngle__bo2__bc,B0_X,B0_ErrX,B0_Y,B0_ErrY,B0_Z,B0_ErrZ,B0_Rho,B0_etap_X,B0_etap_ErrX,B0_etap_Y,B0_etap_ErrY,B0_etap_Z,B0_etap_ErrZ,B0_etap_Rho,B0_etap_eta_X,B0_etap_eta_ErrX,B0_etap_eta_Y,B0_etap_eta_ErrY,B0_etap_eta_Z,B0_etap_eta_ErrZ,B0_etap_eta_Rho,B0_etap_pi0_X,B0_etap_pi0_ErrX,B0_etap_pi0_Y,B0_etap_pi0_ErrY,B0_etap_pi0_Z,B0_etap_pi0_ErrZ,B0_etap_pi0_Rho,B0_etap_pi1_X,B0_etap_pi1_ErrX,B0_etap_pi1_Y,B0_etap_pi1_ErrY,B0_etap_pi1_Z,B0_etap_pi1_ErrZ,B0_etap_pi1_Rho,B0_K_S0_X,B0_K_S0_ErrX,B0_K_S0_Y,B0_K_S0_ErrY,B0_K_S0_Z,B0_K_S0_ErrZ,B0_K_S0_Rho,B0_cosAngleBetweenMomentumAndVertexVector,B0_distance,B0_significanceOfDistance,B0_dr,B0_etap_pi0_dr,B0_etap_pi1_dr,B0_K_S0_dr,B0_VtxPvalue,B0_etap_VtxPvalue,B0_etap_eta_VtxPvalue,B0_etap_pi0_VtxPvalue,B0_etap_pi1_VtxPvalue,B0_K_S0_VtxPvalue,B0_CSMVA,B0_TrCSMVA
0,5.307438,0.015551,1.794601,0.497874,0.003358,0.077336,0.962908,0.002699,1.944490,0.542071,0.010025,-0.576791,1.602125,0.314331,0.325830,1.536826,5.543979,0.028394,5.278986,3.066705,1.227109,2.410350,1.445371,3.212750,2.075677,0.744426,1.626914,1.052338,2.146760,1.225226,0.540652,1.031143,0.381597,1.225226,0.905440,0.198372,0.584079,0.662813,0.905440,0.699465,0.404392,0.475399,0.315775,0.713254,0.334137,0.075672,0.313344,0.087958,0.362115,2.277444,-0.912778,-2.084520,0.091455,2.331229,0.488507,-0.488507,0.568254,0.079900,-0.670724,-0.002149,0.001303,-1.066588e-09,0.000004,-0.014420,0.007413,0.002149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.001742,0.004913,0.001482,0.004186,-0.009292,0.007839,0.002287,0.003826,0.011587,-0.000924,0.002779,-0.019859,0.012930,0.003936,-3.503684,0.056161,-8.044307,0.127328,-0.092195,0.085395,8.774205,-0.977685,0.014580,1.943959,0.002149,0.002287,0.003936,8.774205,6.053036e-04,4.801615e-02,5.644614e-01,5.343583e-01,0.361941,6.989494e-03,0.055560,-2.833126
1,5.229204,0.032386,-1.553946,0.490416,0.014302,-0.503305,0.977696,0.009197,2.178594,0.486767,0.011250,-5.429974,1.770375,0.015410,-0.344991,1.736367,5.520761,-0.039430,5.268792,1.871150,-1.096115,0.942214,-1.188261,2.102231,0.913500,-0.684946,0.393925,-0.458427,1.065188,0.395340,-0.094642,0.313180,-0.221934,0.395340,0.561874,-0.527635,0.037243,-0.189518,0.561874,0.461770,-0.180194,0.266927,-0.330925,0.482401,0.615245,-0.273122,0.314300,-0.452930,0.630877,3.383169,1.111526,-1.287206,2.924628,3.418530,-0.857668,0.857668,-0.527525,0.403934,0.673270,0.006748,0.001236,-2.922411e-09,0.000004,0.045094,0.006744,0.006748,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.006379,0.007717,-0.004306,0.005148,0.063782,0.012361,0.007697,0.000264,0.003745,0.000230,0.003256,0.034392,0.007308,0.000350,6.129555,0.255150,-7.030197,0.321926,16.094391,0.691816,9.327116,0.971280,0.045596,6.678820,0.006748,0.007697,0.000350,9.327117,1.814637e-01,2.815941e-02,5.980884e-08,7.179064e-01,0.958767,9.333900e-01,0.556649,0.227573
2,5.267826,0.012465,-0.938999,0.497261,0.003568,-0.098962,0.957618,0.003712,-0.011401,0.508457,0.009905,-3.977389,1.462492,0.394294,-0.313335,1.373039,5.467072,-0.008629,5.276476,2.782038,0.691355,2.128145,1.653107,2.942240,1.557186,0.276121,1.256326,0.877627,1.6

### Preprocessing the Data
1. Labelling the Data (0 for peaking background, 1 for continuous background and 2 for Signal)
2. Splitting it in train and test data
3. removing the label from testing
4. Scaling it and removing the mean

In [13]:
from random import seed
from random import randint
# seed random number generator
n_seed=1234
seed(n_seed)

In [14]:
Train_Signal['Type'] = 2
Train_Continuous['Type'] = 1
Train_Peaking['Type'] = 0

/home/philipp/anaconda3/envs/BelleAnalysis/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/philipp/anaconda3/envs/BelleAnalysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/philipp/anaconda3/envs/BelleAnalysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [15]:
from sklearn.model_selection import train_test_split

Sum_BS = pd.concat([Train_Signal, Train_Continuous, Train_Peaking]).sample(frac=1)
X = Sum_BS.drop('Type',axis=1)
Y = Sum_BS['Type']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8,random_state=randint(10**6,10**9))

In [16]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
print(ss.fit(X_train))
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

StandardScaler(copy=True, with_mean=True, with_std=True)


In [17]:
from keras.utils import to_categorical
Y_train, Y_test = to_categorical(Y_train, num_classes=3), to_categorical(Y_test, num_classes=3)

Using TensorFlow backend.


In [18]:
#from sklearn.preprocessing import OneHotEncoder
#OHE = OneHotEncoder(sparse=False)
#Y_train, Y_test = OHE.fit_transform(np.array(Y_train).reshape(-1,1)), OHE.fit_transform(np.array(Y_test).reshape(-1,1))

In [19]:
display(Y_train[0])

array([0., 0., 1.], dtype=float32)

## Create the Neural Network

In [20]:
model = keras.Sequential([
    keras.layers.Dense(116, activation=tf.nn.relu),
    keras.layers.Dense(150, activation=tf.nn.relu),
    keras.layers.Dense(3, activation=tf.nn.softmax)
])

W0627 17:08:02.272599 140085325678400 deprecation.py:506] From /home/philipp/anaconda3/envs/BelleAnalysis/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [21]:
model.compile(optimizer='adam', 
              loss='kullback_leibler_divergence',
              metrics=['accuracy'])


In [22]:
X_train.shape

(363152, 116)

In [23]:
model.fit(X_train, Y_train, epochs=25)

W0627 17:08:02.466756 140085325678400 deprecation.py:323] From /home/philipp/anaconda3/envs/BelleAnalysis/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/25
363152/363152 [==============================] - 37s 102us/sample - loss: 0.0630 - acc: 0.9819
Epoch 2/25
363152/363152 [==============================] - 36s 98us/sample - loss: 0.0514 - acc: 0.9855
Epoch 3/25
363152/363152 [==============================] - 36s 99us/sample - loss: 0.0475 - acc: 0.9865
Epoch 4/25
363152/363152 [==============================] - 37s 103us/sample - loss: 0.0455 - acc: 0.9871
Epoch 5/25
363152/363152 [==============================] - 40s 110us/sample - loss: 0.0439 - acc: 0.9875
Epoch 6/25
363152/363152 [==============================] - 35s 97us/sample - loss: 0.0421 - acc: 0.9882
Epoch 7/25
363152/363152 [==============================] - 43s 118us/sample - loss: 0.0407 - acc: 0.9885
Epoch 8/25
363152/363152 [==============================] - 42s 116us/sample - loss: 0.0402 - acc: 0.9888
Epoch 9/25
363152/363152 [==============================] - 35s 97us/sample - loss: 0.0381 - acc: 0.9894
Epoch 10/25
363152/363152 [=======================

2019-06-27 17:08:02.591822: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2019-06-27 17:08:02.702922: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2400000000 Hz
2019-06-27 17:08:02.703602: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556a9295b890 executing computations on platform Host. Devices:
2019-06-27 17:08:02.703648: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-3
OMP: Info #156: KMP_AFFINITY: 4 available OS procs
OMP: Info #1

### Evaluate Model

In [25]:
test_loss, test_acc = model.evaluate(X_test, Y_test)

90789/90789 [==============================] - 4s 44us/sample - loss: 0.0642 - acc: 0.9855
